<img src="images/knapsack.png" width="400px">
- Maximize the sum of the values of the items in the knapsack
- The sum of the weights is less than or equal to the knapsack's capacity

<!---
reference
https:'/'/github.com'/KendallPark'/genetic-algorithm
-->

$$max \sum c_i x_i$$
$$s.t.\sum a_i x_i \le b $$
$$ x_i \in {0,1} \forall i $$

# Datas
source : Pisinger, D. (2005). "Where are the hard knapsack problems?" <i>Computers & Operations Research</i>

In [2]:
def makeProbs(datafile):
    f = open(datafile,'r')
    lines = f.readlines()

    probs = []

    prob = []
    for line in lines:
        line = line.rstrip()

        if line.startswith('-----'):
            probs.append(prob)
            prob = []
        elif line != '':
            prob.append(line)

    return [makeProb(p) for p in probs]
        
def sort_desc_by_a(c, a, sol):
    sorted_idx = sorted(range(len(a)),key=lambda x:a[x], reverse=True)
    new_a = [a[si] for si in sorted_idx]
    new_c = [c[si] for si in sorted_idx]
    new_sol = [sol[si] for si in sorted_idx]
    d = [int(na*0.3) for na in new_a]
    return new_c, new_a, new_sol, d

def makeProb(dat):
    info = []
    for dp in dat[1:5]:
        info.append(float(dp.split()[1]))
    
    item = []
    for d in dat[5:]:
        a = d.split(',')
        a = map(int,a)
        item.append(a)
    
    c = [it[1] for it in item]
    a = [it[2] for it in item]
    sol = [it[3] for it in item]
    sorted_c, sorted_a, sorted_sol, sorted_d = sort_desc_by_a(c,a,sol)
    a_d = [sorted_a[i]+sorted_d[i] for i in range(len(a))]
    
    return {
        'n': int(info[0]),
        'b': int(info[1]),
        'optval': int(info[2]),
        'c' : sorted_c, 
        'a' : sorted_a,
        'sol' : sorted_sol,
        'd' : sorted_d
        
    }

In [3]:
probs = makeProbs('./datasets/knapPI_1_50_1000.csv')

# Genetic Algorithm

1. [Initialization] popSize개의 chromosomes(individuals) 생성
2. [Fitness] (1)에서 생성된 각 individual의 fitness value 구하기
3. [New population] 다음 과정에 따라 새로운 population 생성
    1. [Selection] fitness value가 가장 좋은 두개의 parent individuals 선택. (the better fitness, the bigger chance to be selected)
    2. [Crossover] (3-A)에서 선택한 두 개의 parent individuals로 child individual 생성. (single-point crossover)
    3. [Mutation] (3-2)의 individual의 각 gene을 주어진 mutation 확률로 돌연변이 시키기. (0->1, 1->0)
4. [Replace] Use new generated population for a further run of algorithm
5. [Test] 종료조건 만족하면 과정을 멈추고 현재 population의 가장 좋은 솔루션 리턴. 
6. [Loop] (2)로 돌아가기

In [73]:
import random
import numpy as np
import time

In [44]:
def knapsack(V, W, MAX, popSize, mut, maxGen, percent):
    generation = 0
    pop = generate(V, popSize) # 1.Initialization
    fitness = getFitness(pop, V, W, MAX) # 2.Fitness
    while(not test(fitness, percent) and generation < maxGen): # 5.Test : 주어진 percent만큼 convergence 되거나, maxGen만큼 generation 돌았거나
        generation += 1
        pop = newPopulation(pop, fitness, mut) # 3.New population,  4. Replace
        fitness = getFitness(pop, V, W, MAX)
    return selectElite(pop, fitness)

# initial population 생성
def generate(V, popSize):
    length = len(V)
    pop = [[random.randint(0,1) for i in range(length)] for j in range(popSize)] # 각 gene 0,1 중 랜덤하게 선택
    # pop
    # [[1,0,0,1],
    #  [0,1,1,1]...]
    return pop

# 각 individual의 sum(c) 값 return
def getFitness(pop, V, W, MAX):
    fitness = []
    for i in range(len(pop)): #100
        weight = 0
        volume = MAX+1
        while (volume > MAX):
            weight = 0
            volume = 0
            ones = []
            for j in range(len(pop[i])): #아이템개수만큼 돌기 (i번째population)
                if pop[i][j] == 1: #선택하면
                    volume += V[j]
                    weight += W[j]
                    ones += [j]
            if volume > MAX: #용량초과되면 선택(1)된것 중 하나 안선택(0)으로 바꿔
                pop[i][ones[random.randint(0, len(ones)-1)]] = 0
        fitness += [weight]
    return fitness


def newPopulation(pop, fit, mut):
    popSize = len(pop)
    newPop = []
    newPop += [selectElite(pop, fit)]
    while(len(newPop) < popSize):
        (mate1, mate2) = select(pop, fit) # 3-A.Selection
        newPop += [mutate(crossover(mate1, mate2), mut)] # 3-B.Crossover, 3-C.Mutation
    return newPop

# population 중 fit 값이 가장 좋은 거 하나 return
def selectElite(pop, fit):
    elite = 0
    for i in range(len(fit)):
        if fit[i] > fit[elite]:
            elite = i
    return pop[elite]

# fit이 가장 좋은 두개 gene return 
def select(pop, fit):
    size = len(pop)
    totalFit = sum(fit)
    lucky = random.randint(0, totalFit)
    tempSum = 0
    mate1 = []
    fit1 = 0
    for i in range(size):
        tempSum += fit[i]
        if tempSum >= lucky:
            mate1 = pop.pop(i) # tempsum이 lucky를 넘게 한 gene
            fit1 = fit.pop(i) #         ( = )           gene의 fit
            break
    
    tempSum = 0
    lucky = random.randint(0, sum(fit)) 
    for i in range(len(pop)):
        tempSum += fit[i]
        if tempSum >= lucky:
            mate2 = pop[i]
            pop += [mate1] # pop 복구
            fit += [fit1]  # fit 복구
            return (mate1, mate2)

# crossover할 포인트 random으로 선택 후 crossover // 하나의 child gene return.
def crossover(mate1, mate2):
    lucky = random.randint(0, len(mate1)-1)
    return mate1[:lucky]+mate2[lucky:]
  
# 1/mutate 확률로 1->0, 0->1
def mutate(gene, mutate):
    for i in range(len(gene)):
        lucky = random.randint(1, mutate)
        if lucky == 1:
            gene[i] = bool(gene[i])^1 #1->0, 0->1
    return gene

# fit 중 최빈값의 비율이 rate 이상이면 true.
def test(fit, rate):
    maxCount = mode(fit)
#     print 'mode rate',float(maxCount)/float(len(fit)), rate
    if float(maxCount)/float(len(fit)) >= rate:
        return True
    else:
        return False

# 최빈값
def mode(fit): 
    values = set(fit) #중복 제거
    maxCount = 0
    for i in values:
        if maxCount < fit.count(i):
            maxCount = fit.count(i)
    return maxCount #fitness 값 중, 가장 많은 값의 개수를 리턴

def answer(solution, A, C, b):
    sum_of_weights = np.dot(solution,A)
    if sum_of_weights<=b:
        feasible=True
    else:
        feasible=False
    optval =  np.dot(solution,C)
    
    if feasible:
        print 'GA Feasible'
        print 'GA best value : ',optval
    else:
        print 'GA Infeasible'

### excercise 1

In [45]:
size = 4
A = [10,5,5,20] # a
C = [40,20,10,50] # c
b = 20
popSize = 100
mut = 100 # 1/mut 확률로 mutation
maxGen = 10*size
percent = 0.9
final_solution = knapsack(A, C, b, popSize, mut, maxGen, percent) 
print final_solution
answer(final_solution, A, C, b)

[1, 1, 1, 0]
GA Feasible
GA best value :  70


# DP algorithm

In [61]:
def dp(r, lmd, c, a):
    k = np.zeros((lmd+1, r+1))
    
    for i in range(lmd+1):
        for j in range(1,r+1):
            if i<a[j]: pick = 0
            else: pick = c[j]
            k[i][j] = max(k[i][j-1], pick+k[i-a[j]][j-1])
    return k[-1][-1]

In [77]:
problem = probs[1]
n = problem['n']
A = problem['a']
C = problem['c']
b = problem['b']
sol = problem['sol']
optval = problem['optval']

popSize = 100
mut = 100 # 1/mut 확률로 mutation
maxGen = 10*size
percent = 0.9

ga_time_before = time.time()
final_solution = knapsack(A, C, b, popSize, mut, maxGen, percent)
ga_time = time.time()-ga_time_before

dp_time_before = time.time()
dp_opt_val = dp(n,b,[0]+C,[0]+A)
dp_time = time.time()-dp_time_before

# print final_solution
answer(final_solution, A, C, b)
print 'DP Optimal value : ',dp_opt_val
# print sol
print 'Optimal value : ', optval
print ''
print 'GA time : ',ga_time
print 'DP time : ',dp_time

GA Feasible
GA best value :  4970
DP Optimal value :  5847.0
Optimal value :  5847

GA time :  0.461004018784
DP time :  0.0605928897858
